In [6]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

dtype = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str
}

df_customer = pd.read_csv("customer_eng.csv", dtype=dtype)
df_category = pd.read_csv("category_eng.csv", dtype=dtype)
df_product  = pd.read_csv("product.csv", dtype=dtype)
df_receipt  = pd.read_csv("receipt.csv", dtype=dtype)
df_store    = pd.read_csv("store_eng.csv", dtype=dtype)
df_geocode  = pd.read_csv("geocode_eng.csv", dtype=dtype)


In [6]:
#Từ dữ liệu biên nhận (df_receipt), hiển thị 10 bản ghi đầu tiên 
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


In [7]:
#và kiểm tra xem loại dữ liệu nào được lưu trữ trong các cột.
df_receipt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104681 entries, 0 to 104680
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   sales_ymd       104681 non-null  int64 
 1   sales_epoch     104681 non-null  int64 
 2   store_cd        104681 non-null  object
 3   receipt_no      104681 non-null  int64 
 4   receipt_sub_no  104681 non-null  int64 
 5   customer_id     104681 non-null  object
 6   product_cd      104681 non-null  object
 7   quantity        104681 non-null  int64 
 8   amount          104681 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 7.2+ MB


In [14]:
#Từ dữ liệu biên nhận (df_receipt), chỉ định các cột theo thứ tự sau và hiển thị 10 bản ghi: ngày bán hàng (sales_ymd), ID khách hàng (customer_id), mã sản phẩm (product_cd), số tiền bán hàng (amount).
df_receipt [["sales_ymd","customer_id","product_cd","amount"]].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


In [ ]:
#Từ dữ liệu biên nhận (df_receipt), chỉ định các cột theo thứ tự sau và hiển thị 10 bản ghi: ngày bán hàng (sales_ymd), ID khách hàng (customer_id), mã sản phẩm (product_cd), số tiền bán hàng (amount). Tuy nhiên, đổi tên cột sales_ymd thành sales_date trước khi hiển thị.
df_receipt1= df_receipt.rename(columns= {"sales_ymd": "sales_date"})
df_receipt1[["sales_date", "customer_id", "product_cd","amount"]].head(10)

#cách 2:
(
    df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]]
    .rename(columns={"sales_ymd": "sales_date"})
).head()

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


In [20]:
#ừ dữ liệu biên nhận (df_receipt), chỉ định các cột theo thứ tự sau và trích xuất dữ liệu đáp ứng các điều kiện dưới đây: ngày bán hàng (sales_ymd), ID khách hàng (customer_id), mã sản phẩm (product_cd), số tiền bán hàng (amount).

# ID khách hàng (customer_id) là "CS018205000001"
df_receipt1[["sales_date", "customer_id", "product_cd","amount"]].query("customer_id== 'CS018205000001'")

,sales_date,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [22]:
#Từ dữ liệu biên nhận (df_receipt), chỉ định các cột theo thứ tự sau và trích xuất dữ liệu đáp ứng tất cả các điều kiện dưới đây: ngày bán hàng (sales_ymd), ID khách hàng (customer_id), mã sản phẩm (product_cd), số tiền bán hàng (amount).
#ID khách hàng (customer_id) là "CS018205000001"
#Số tiền bán hàng (amount) lớn hơn hoặc bằng 1000
df_receipt1[["sales_date", "customer_id", "product_cd","amount"]].query("(customer_id=='CS018205000001') & (amount >= 1000)")

,sales_date,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [26]:
#Từ dữ liệu biên nhận (df_receipt), chỉ định các cột theo thứ tự sau và trích xuất dữ liệu đáp ứng tất cả các điều kiện dưới đây: ngày bán hàng (sales_ymd), ID khách hàng (customer_id), mã sản phẩm (product_cd), số lượng bán hàng (quantity), số tiền bán hàng (amount).

#ID khách hàng (customer_id) là "CS018205000001"
#Số tiền bán hàng (amount) lớn hơn hoặc bằng 1000 hoặc số lượng bán hàng (quantity) lớn hơn hoặc bằng 5
df_receipt1[["sales_date", "customer_id", "product_cd","quantity","amount"]].query("(customer_id=='CS018205000001') & (amount >= 1000 |quantity >= 5)")

,sales_date,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


In [28]:
#P-007: Từ dữ liệu biên nhận (df_receipt), chỉ định các cột theo thứ tự sau và trích xuất dữ liệu đáp ứng tất cả các điều kiện dưới đây: ngày bán hàng (sales_ymd), ID khách hàng (customer_id), mã sản phẩm (product_cd), số tiền bán hàng (amount).

#ID khách hàng (customer_id) là "CS018205000001"
#Số tiền bán hàng (amount) lớn hơn hoặc bằng 1000 và nhỏ hơn hoặc bằng 2000
df_receipt1[["sales_date", "customer_id", "product_cd","amount"]].query("customer_id== 'CS018205000001' & (1000 <= amount <= 2000)")

,sales_date,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


In [30]:
#Từ dữ liệu biên nhận (df_receipt), chỉ định các cột theo thứ tự sau và trích xuất dữ liệu đáp ứng tất cả các điều kiện dưới đây: ngày bán hàng (sales_ymd), ID khách hàng (customer_id), mã sản phẩm (product_cd), số tiền bán hàng (amount).
#ID khách hàng (customer_id) là "CS018205000001"
#Mã sản phẩm (product_cd) không phải là "P071401019"
df_receipt1[["sales_date", "customer_id", "product_cd","amount"]].query("(customer_id =='CS018205000001') & (product_cd != 'P071401019')")

,sales_date,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [36]:
#Thay đổi điều kiện lọc dưới đây và thay thế từ khóa OR thành AND trong truy vấn xuất dữ liệu.

#df_store.query('not(prefecture_cd == "13" | floor_area > 900)')
df_store.query("not (prefecture_cd == '13' and floor_area > 900)")

,store_cd,store_name,prefecture_cd,prefecture,address,tel_no,longitude,latitude,floor_area
0,S12014,Chigakadai store,12,Chiba prefecture,"1-chome, Chigusadai, Inage-ku, Chiba-shi, Chiba",043-123-4003,140.1180,35.63559,1698.0
2,S14010,Kikuna store,14,Kanagawa Prefecture,"Kikuna 1-chome, Kohoku-ku, Yokohama-shi, Kanagawa",045-123-4032,139.6326,35.50049,1732.0
3,S14033,Akuwa store,14,Kanagawa Prefecture,"Akuwa Nishi 1-chome, Seya-ku, Yokohama-shi, Ka...",045-123-4043,139.4961,35.45918,1495.0
4,S14036,Sagamihara Central Store,14,Kanagawa Prefecture,"2-chome, Chuo, Sagamihara City, Kanagawa Prefe...",042-123-4045,139.3716,35.57327,1679.0
7,S14040,Nagatsuta store,14,Kanagawa Prefecture,"Nagatsuda Minamidai 5-chome, Midori-ku, Yokoha...",045-123-4046,139.4994,35.52398,1548.0
9,S14050,Akuwa Nishi store,14,Kanagawa Prefecture,"Akuwa Nishi 1-chome, Seya-ku, Yokohama-shi, Ka...",045-123-4053,139.4961,35.45918,1830.0
12,S14028,Futatsu Bridge store,14,Kanagawa Prefecture,"Futatsuhashi-cho, Seya-ku, Yokohama-shi, Kanagawa",045-123-4042,139.4963,35.46304,1574.0
16,S14012,Honmaki Wada store,14,Kanagawa Prefecture,"Honmaki Wada, Naka-ku, Yokohama-shi, Kanagawa",045-123-4034,139.6582,35.42156,1341.0
18,S14046,Kitayamada store,14,Kanagawa Prefecture,"Kitamada 1-chome, Tsuzuki-ku, Yokohama-shi, Ka...",045-123-4049,139.5916,35.56189,831.0
19,S14022,Zushi store,14,Kanagawa Prefecture,"Zushi 1-chome, Zushi City, Kanagawa Prefecture",046-123-4036,139.5789,35.29642,1838.0


In [44]:
#Từ dữ liệu cửa hàng (df_store), trích xuất và hiển thị 10 bản ghi với mã cửa hàng (store_cd) bắt đầu bằng 'S14'.
(df_store.query("store_cd.str.startswith('S14')")).head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,tel_no,longitude,latitude,floor_area
2,S14010,Kikuna store,14,Kanagawa Prefecture,"Kikuna 1-chome, Kohoku-ku, Yokohama-shi, Kanagawa",045-123-4032,139.6326,35.50049,1732.0
3,S14033,Akuwa store,14,Kanagawa Prefecture,"Akuwa Nishi 1-chome, Seya-ku, Yokohama-shi, Ka...",045-123-4043,139.4961,35.45918,1495.0
4,S14036,Sagamihara Central Store,14,Kanagawa Prefecture,"2-chome, Chuo, Sagamihara City, Kanagawa Prefe...",042-123-4045,139.3716,35.57327,1679.0
7,S14040,Nagatsuta store,14,Kanagawa Prefecture,"Nagatsuda Minamidai 5-chome, Midori-ku, Yokoha...",045-123-4046,139.4994,35.52398,1548.0
9,S14050,Akuwa Nishi store,14,Kanagawa Prefecture,"Akuwa Nishi 1-chome, Seya-ku, Yokohama-shi, Ka...",045-123-4053,139.4961,35.45918,1830.0
12,S14028,Futatsu Bridge store,14,Kanagawa Prefecture,"Futatsuhashi-cho, Seya-ku, Yokohama-shi, Kanagawa",045-123-4042,139.4963,35.46304,1574.0
16,S14012,Honmaki Wada store,14,Kanagawa Prefecture,"Honmaki Wada, Naka-ku, Yokohama-shi, Kanagawa",045-123-4034,139.6582,35.42156,1341.0
18,S14046,Kitayamada store,14,Kanagawa Prefecture,"Kitamada 1-chome, Tsuzuki-ku, Yokohama-shi, Ka...",045-123-4049,139.5916,35.56189,831.0
19,S14022,Zushi store,14,Kanagawa Prefecture,"Zushi 1-chome, Zushi City, Kanagawa Prefecture",046-123-4036,139.5789,35.29642,1838.0
20,S14011,Hiyoshi Honmachi store,14,Kanagawa Prefecture,"4-chome Honcho, Kohoku-ku, Yokohama-shi, Kanagawa",045-123-4033,139.6316,35.54655,890.0


In [48]:
#Từ dữ liệu khách hàng (df_customer), trích xuất và hiển thị 10 bản ghi với mã ID khách hàng (customer_id) kết thúc bằng '1'.
df_customer.query("customer_id.str.endswith('1')",engine='python').head(10)


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS033515000071,Yuki Suwa,1,Male,1966-04-30,52,246-0031,"Seya -ku, Yokohama -shi, Kanagawa",S14033,20151129,D-20100930-D
7,CS022515000271,Riho Nakagawa,1,Male,1966-08-27,52,249-0005,"Sakurayama, Zushi City, Kanagawa Prefecture",S14022,20181010,0-00000000-0
10,CS015212000021,Fujimura 7th,1,Male,1997-03-20,22,136-0071,"Kameido, Koto -ku, Tokyo",S13015,20150812,0-00000000-0
17,CS007502000011,Koji Asakura,0,Female,1962-07-28,56,285-0854,"Kamiza, Sakura City, Chiba Prefecture",S12007,20150328,0-00000000-0
20,CS004415000861,Ryoko Omori,1,Male,1974-01-28,45,165-0032,"Saginomiya, Nakano -ku, Tokyo",S13004,20170525,0-00000000-0
36,CS007702000011,History of Takeshi Nagai,0,Female,1941-04-22,77,285-0866,"Usuidai, Sakura City, Chiba Prefecture",S12007,20150609,0-00000000-0
37,CS029503000041,Nakahara Subaru,0,Female,1966-02-13,53,134-0084,"Higashi Kasai, Edogawa -ku, Tokyo",S12029,20150222,0-00000000-0
60,CS005713000021,Ruria Noda,1,Male,1943-10-03,75,167-0022,"Suginami -ku, Tokyo Shimoigusa",S13005,20150317,0-00000000-0
76,CS004603000051,Taro Matsutani,0,Female,1952-12-11,66,165-0032,"Saginomiya, Nakano -ku, Tokyo",S13004,20161014,0-00000000-0
77,CS003314000081,Takakura sandbags,1,Male,1984-06-27,34,201-0012,"Komae -shi, Tokyo Ikuwaki",S13003,20150324,6-20100917-7


In [51]:
#Từ dữ liệu cửa hàng (df_store), trích xuất và hiển thị 10 bản ghi với địa chỉ (address) có chứa từ 'Tokyo'.
df_store.query("address.str.contains('Tokyo')",engine='python').head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,tel_no,longitude,latitude,floor_area
1,S13002,Kokubunji Temple,13,Tokyo,"Honda 2-chome, Kokubunji City, Tokyo",042-123-4008,139.4802,35.70566,1735.0
5,S13051,Itabashi Ohara store,13,Tokyo,"Ohara-cho, Itabashi-ku, Tokyo",03-0123-4029,139.6980,35.76788,1045.0
6,S13015,Southern sand store,13,Tokyo,"2-chome, Koto-ku, Tokyo 2-chome",03-0123-4014,139.8229,35.67066,1337.0
8,S13044,Minami -Rokugo store,13,Tokyo,"Minami Rokugo 2-chome, Ota-ku, Tokyo",03-0123-4028,139.7207,35.54604,1379.0
10,S13019,Komorone store,13,Tokyo,"Komorine 1-chome, Itabashi-ku, Tokyo",03-0123-4018,139.6795,35.74468,1004.0
11,S13052,Morino store,13,Tokyo,"Morino Sanchome, Machida City, Tokyo",042-123-4030,139.4383,35.55293,1087.0
13,S13031,Hatsudai store,13,Tokyo,"Hatsudai 2-chome, Shibuya-ku, Tokyo",03-0123-4020,139.6862,35.67616,986.0
14,S13038,Higashikasai store,13,Tokyo,"Tokyo Kasai 9-chome, Edogawa-ku, Tokyo",03-0123-4024,139.8826,35.65366,1184.0
15,S13001,Naka -Rokugo store,13,Tokyo,"2-chome, Nakarokugo, Ota-ku, Tokyo",03-1234-4007,139.7132,35.55135,1796.0
17,S13004,Sagimiya store,13,Tokyo,"Saginomiya Sanchome, Nakano-ku, Tokyo",03-0123-4010,139.6421,35.72348,947.0


In [55]:
#Từ dữ liệu khách hàng (df_customer), trích xuất và hiển thị 10 bản ghi với mã trạng thái (status_cd) bắt đầu bằng chữ cái từ 'A' đến 'F'.
df_customer.query("status_cd.str.contains('^[A-F]')",engine="python").head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS019415000154,Yuki Umemura,9,Unknown,1971-07-08,47,179-0084,"Hikawadai, Nerima -ku, Tokyo",S13019,20150506,E-20101014-E
4,CS033515000071,Yuki Suwa,1,Male,1966-04-30,52,246-0031,"Seya -ku, Yokohama -shi, Kanagawa",S14033,20151129,D-20100930-D
16,CS024415000146,Sachiko Hosaka,1,Male,1977-10-31,41,214-0038,"Ikuta, Tama -ku, Kawasaki City, Kanagawa Prefe...",S14024,20150518,D-20101026-F
45,CS019415000084,Honzan Mami,1,Male,1971-12-20,47,176-0003,"Hazawa, Nerima -ku, Tokyo",S13019,20151109,C-20101012-E
53,CS019214000008,Reina Okuno,1,Male,1991-07-01,27,171-0044,"Chihaya, Toshima -ku, Tokyo",S13019,20150102,D-20100714-A
54,CS002515000256,Yoko Kikuchi,1,Male,1964-10-06,54,185-0013,"Kokubunji -shi, Tokyo Nishiki Kokubo",S13002,20161201,A-20100303-7
63,CS017315000023,Reina Uchino,1,Male,1987-02-21,32,166-0015,"Narita Higashi, Suginami -ku, Tokyo",S13017,20141209,C-20090424-5
70,CS030415000034,Tamaki Nanko,1,Male,1969-01-28,50,272-0025,"Owada, Ichikawa City, Chiba Prefecture",S12030,20150501,F-20101020-F
72,CS025214000016,Furukawa Erika,1,Male,1995-10-06,23,245-0018,"Kamiiida -cho, Izumi -ku, Yokohama -shi, Kanagawa",S14025,20150825,D-20101029-F
101,CS021115000004,Hikaru Kurata,1,Male,2003-01-19,16,259-1111,"Nishi -Tomioka, Isehara City, Kanagawa Prefecture",S14021,20150212,E-20100925-E


In [56]:
#Từ dữ liệu khách hàng (df_customer), trích xuất và hiển thị 10 bản ghi với mã trạng thái (status_cd) kết thúc bằng các số từ '1' đến '9'.
df_customer.query("status_cd.str.contains('[1-9]$')").head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS008514000047,Nozomi Ishikawa,1,Male,1962-04-03,56,201-0004,"Iwato Kita, Komae City, Tokyo",S13008,20150423,6-20100613-8
3,CS004615000296,Yu Tateishi,1,Male,1950-02-23,69,165-0027,"Nakano -ku, Tokyo Nomo",S13004,20160618,4-20080701-1
8,CS020412000046,An apricot of Kurosaki,1,Male,1971-12-09,47,174-0056,"Shimura, Itabashi -ku, Tokyo",S13020,20150820,1-20100516-3
11,CS031514000047,Reiko Haraguchi,1,Male,1963-07-26,55,151-0064,"Uehara, Shibuya -ku, Tokyo",S13031,20150927,8-20090607-6
12,CS017513000183,Kyoko Suenaga,1,Male,1965-05-05,53,165-0027,"Nakano -ku, Tokyo Nomo",S13017,20151101,1-20090203-3
15,CS044415000024,Chikako Sugino,1,Male,1977-06-02,41,144-0052,"Kamata, Ota -ku, Tokyo",S13044,20180519,6-20101016-9
24,CS025512000055,Mayuko Mizuguchi,1,Male,1966-01-15,53,242-0024,"Fukuda, Yamato City, Kanagawa Prefecture",S14025,20150108,4-20090705-4
34,CS011315000042,Aya Inagaki,1,Male,1982-02-04,37,223-0061,"Hiyoshi, Kohoku -ku, Yokohama -shi, Kanagawa",S14011,20150618,1-20080219-3
41,CS001315000186,Mikako Tomita,1,Male,1979-11-13,39,144-0055,"Nakarokugo, Ota -ku, Tokyo",S13001,20150503,3-20080412-1
49,CS026615000130,Rie Matsunaga,1,Male,1951-12-19,67,251-0043,"Tsujido Motomachi, Fujisawa City, Kanagawa Pre...",S14026,20150128,8-20090727-5


In [58]:
#Từ dữ liệu khách hàng (df_customer), trích xuất và hiển thị 10 bản ghi với mã trạng thái (status_cd) bắt đầu bằng chữ cái từ 'A' đến 'F' và kết thúc bằng các số từ '1' đến '9'.
df_customer.query("status_cd.str.contains('^[A-F].*[1-9]$')").head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
54,CS002515000256,Yoko Kikuchi,1,Male,1964-10-06,54,185-0013,"Kokubunji -shi, Tokyo Nishiki Kokubo",S13002,20161201,A-20100303-7
63,CS017315000023,Reina Uchino,1,Male,1987-02-21,32,166-0015,"Narita Higashi, Suginami -ku, Tokyo",S13017,20141209,C-20090424-5
163,CS022415000100,Misako Sunagawa,1,Male,1976-07-10,42,249-0007,"Shinjuku, Zushi City, Kanagawa Prefecture",S14022,20150323,B-20091009-9
167,CS038314000035,Kyoko Okunuki,1,Male,1979-04-09,39,134-0084,"Higashi Kasai, Edogawa -ku, Tokyo",S13038,20151015,A-20100917-9
239,CS025512000143,Yoshino Kikuchi,1,Male,1962-11-24,56,242-0023,"Shibuya, Yamato City, Kanagawa Prefecture",S14025,20160306,B-20090802-5
282,CS010513000187,Asami Matsukawa,1,Male,1966-09-01,52,222-0032,"Samido -cho, Kohoku -ku, Yokohama -shi, Kanagawa",S14010,20150710,D-20100808-2
319,CS020212000014,Matsui Natsuzora,1,Male,1991-05-08,27,332-0015,"Kawaguchi, Kawaguchi City, Saitama Prefecture",S13020,20150707,E-20100524-7
432,CS019215000029,Mayuko Sawa,1,Male,1994-10-14,24,176-0002,"Sakuradai, Nerima -ku, Tokyo",S13019,20151203,A-20100622-9
520,CS015414000138,Erika Kusano,1,Male,1972-07-29,46,136-0076,"Minami sand in Koto -ku, Tokyo",S13015,20150205,D-20100504-9
524,CS035615000040,Daigoro Kawanishi,9,Unknown,1957-02-13,62,158-0098,"Kamimariga, Setagaya -ku, Tokyo",S13035,20150909,E-20100830-6


In [59]:
#Từ dữ liệu cửa hàng (df_store), trích xuất và hiển thị 10 bản ghi với số điện thoại (tel_no) tuân theo qui tắc xxx-xxx-xxxx. Trong đó x là 1 số tự nhiên từ 0-9.
df_store.query("tel_no.str.contains(r'^[0-9]{3}-[0-9]{3}-[0-9]{4}$')").head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,tel_no,longitude,latitude,floor_area
0,S12014,Chigakadai store,12,Chiba prefecture,"1-chome, Chigusadai, Inage-ku, Chiba-shi, Chiba",043-123-4003,140.1180,35.63559,1698.0
1,S13002,Kokubunji Temple,13,Tokyo,"Honda 2-chome, Kokubunji City, Tokyo",042-123-4008,139.4802,35.70566,1735.0
2,S14010,Kikuna store,14,Kanagawa Prefecture,"Kikuna 1-chome, Kohoku-ku, Yokohama-shi, Kanagawa",045-123-4032,139.6326,35.50049,1732.0
3,S14033,Akuwa store,14,Kanagawa Prefecture,"Akuwa Nishi 1-chome, Seya-ku, Yokohama-shi, Ka...",045-123-4043,139.4961,35.45918,1495.0
4,S14036,Sagamihara Central Store,14,Kanagawa Prefecture,"2-chome, Chuo, Sagamihara City, Kanagawa Prefe...",042-123-4045,139.3716,35.57327,1679.0
7,S14040,Nagatsuta store,14,Kanagawa Prefecture,"Nagatsuda Minamidai 5-chome, Midori-ku, Yokoha...",045-123-4046,139.4994,35.52398,1548.0
9,S14050,Akuwa Nishi store,14,Kanagawa Prefecture,"Akuwa Nishi 1-chome, Seya-ku, Yokohama-shi, Ka...",045-123-4053,139.4961,35.45918,1830.0
11,S13052,Morino store,13,Tokyo,"Morino Sanchome, Machida City, Tokyo",042-123-4030,139.4383,35.55293,1087.0
12,S14028,Futatsu Bridge store,14,Kanagawa Prefecture,"Futatsuhashi-cho, Seya-ku, Yokohama-shi, Kanagawa",045-123-4042,139.4963,35.46304,1574.0
16,S14012,Honmaki Wada store,14,Kanagawa Prefecture,"Honmaki Wada, Naka-ku, Yokohama-shi, Kanagawa",045-123-4034,139.6582,35.42156,1341.0


In [60]:
#P-017: Từ dữ liệu khách hàng (df_customer), sắp xếp và hiển thị 10 bản ghi đầu tiên theo thứ tự tăng dần của ngày sinh (birth_day).
df_customer.sort_values('birth_day').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
401,CS017811000003,Yuu Kageyama,1,Male,1930-02-27,89,166-0001,"Asaya Kita, Suginami -ku, Tokyo",S13017,20150224,0-00000000-0
258,CS030802000009,Shingo Shogo,0,Female,1930-08-31,88,272-0803,"Ichikawa -shi, Chiba",S12030,20180506,0-00000000-0
927,CS038813000006,Miyuki Toshima,1,Male,1931-01-07,88,134-0084,"Higashi Kasai, Edogawa -ku, Tokyo",S13038,20150603,0-00000000-0
640,CS005812000025,Ryo Nagase,1,Male,1931-10-06,87,176-0022,"Mukoyama, Nerima -ku, Tokyo",S13005,20170524,0-00000000-0
200,CS024815000023,Mayuko Takenaka,1,Male,1931-11-25,87,214-0035,"Nagasawa, Tama -ku, Kawasaki City, Kanagawa Pr...",S14024,20151030,2-20080925-4
812,CS034813000001,Yuko Yasuko,1,Male,1932-03-03,87,216-0001,"Nogawa, Miyamae -ku, Kawasaki City, Kanagawa P...",S14034,20150625,0-00000000-0
133,CS039813000020,Toshima light,1,Male,1932-03-29,87,167-0034,"Momoi, Suginami -ku, Tokyo",S13039,20150630,0-00000000-0
129,CS002815000035,Ema Kawano,1,Male,1932-04-17,86,185-0024,"Izumi -cho, Kokubunji, Tokyo",S13002,20161227,3-20090908-2
399,CS032815000020,Natsuki Doi,1,Male,1932-06-14,86,144-0056,"Nishi -Rokugo, Ota -ku, Tokyo",S13032,20151104,0-00000000-0
128,CS007813000023,Airi Mizoguchi,1,Male,1932-06-28,86,285-0843,"Nakashizu, Sakura City, Chiba Prefecture",S12007,20150501,0-00000000-0


In [61]:
#P-018: Từ dữ liệu khách hàng (df_customer), sắp xếp và hiển thị 10 bản ghi đầu tiên theo thứ tự giảm dần của ngày sinh (birth_day).
df_customer.sort_values('birth_day', ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
134,CS004112000011,Miyu Tamaki,1,Male,2007-01-03,12,161-0031,"Nishimoho, Shinjuku -ku, Tokyo",S13004,20160615,0-00000000-0
516,CS021102000003,Nakahara,0,Female,2006-05-17,12,254-0004,"Kobujima, Hiratsuka City, Kanagawa Prefecture",S14021,20180510,0-00000000-0
560,CS032115000007,Tomono Noguchi,1,Male,2006-04-11,12,144-0056,"Nishi -Rokugo, Ota -ku, Tokyo",S13032,20150428,B-20100822-A
44,CS019114000002,Mami Morii,1,Male,2004-12-17,14,171-0041,"Chikawa, Toshima -ku, Tokyo",S13019,20150507,0-00000000-0
703,CS037112000007,Sayaka Temple,1,Male,2004-01-20,15,101-0031,"Higashi Kanda, Chiyoda -ku, Tokyo",S13037,20151108,0-00000000-0
250,CS032115000006,Koizumi High History,9,Unknown,2003-11-14,15,144-0056,"Nishi -Rokugo, Ota -ku, Tokyo",S13032,20150223,6-20100725-A
101,CS021115000004,Hikaru Kurata,1,Male,2003-01-19,16,259-1111,"Nishi -Tomioka, Isehara City, Kanagawa Prefecture",S14021,20150212,E-20100925-E
512,CS030112000001,Haruki Yamauchi,1,Male,2002-01-26,17,272-0814,"Takaishijin, Ichikawa City, Chiba Prefecture",S12030,20150620,0-00000000-0
867,CS011412000021,Riho Tamada,1,Male,1999-10-21,19,165-0022,"Ekoda, Nakano -ku, Tokyo",S14011,20150526,4-20090412-3
938,CS028213000058,Mayuko Terashima,1,Male,1997-12-17,21,246-0025,"Akuwanishi, Seya -ku, Yokohama -shi, Kanagawa",S14028,20180806,0-00000000-0


In [65]:
#P-019: Từ dữ liệu biên nhận (df_receipt), xếp hạng số tiền bán hàng (amount) theo thứ tự giảm dần và hiển thị 10 bản ghi đầu tiên. Đính kèm các cột ID khách hàng (customer_id) và số tiền bán hàng (amount) với hạng tương ứng. Nếu số tiền bán hàng (amount) bằng nhau, hạng sẽ giống nhau.
df_receipt2= df_receipt[["amount"]].rank(method="min", ascending=False).rename(columns={"amount":"ranking"}).astype("int")
df_receipt2= pd.concat([df_receipt[["customer_id","amount"]],df_receipt2], axis=1)
df_receipt2.sort_values("ranking").head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1
62317,ZZ000000000000,6800,2
54095,CS028605000002,5780,3
72747,ZZ000000000000,5480,4
10320,ZZ000000000000,5480,4
4632,CS015515000034,5480,4
97294,CS021515000089,5440,7
28304,ZZ000000000000,5440,7
11275,CS017414000114,5280,9
55795,CS032414000072,5280,9


In [7]:
#P-020: Từ dữ liệu biên nhận (df_receipt), xếp hạng số tiền bán hàng (amount) theo thứ tự tăng dần và hiển thị 10 bản ghi đầu tiên. Đính kèm các cột ID khách hàng (customer_id) và số tiền bán hàng (amount) với hạng tương ứng. Trong trường hợp số tiền bán hàng (amount) bằng nhau hạng sẽ không được |giống nhau.
df_tmp = df_receipt[["amount"]].rank(method="first", ascending=False).rename(columns={"amount":"ranking"}).astype("int")
df_tmp = pd.concat([df_receipt[["customer_id", "amount"]], df_tmp], axis = 1)
df_tmp.sort_values("ranking").head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1
62317,ZZ000000000000,6800,2
54095,CS028605000002,5780,3
4632,CS015515000034,5480,4
10320,ZZ000000000000,5480,5
72747,ZZ000000000000,5480,6
28304,ZZ000000000000,5440,7
97294,CS021515000089,5440,8
596,CS015515000083,5280,9
11275,CS017414000114,5280,10


In [8]:
#P-021: Từ dữ liệu biên nhận (df_receipt), đếm số lượng bản ghi.
len(df_receipt)

104681

In [ ]:
#P-022: Từ dữ liệu biên nhận (df_receipt), đếm số lượng các giá trị duy nhất của ID khách hàng (customer_id).
len(df_receipt["customer_id"].unique())
#df_receipt["customer_id"].nunique()

8307

In [ ]:
#P-023: Từ dữ liệu biên nhận (df_receipt), tính tổng số tiền bán hàng (amount) và số lượng bán hàng (quantity) cho mỗi cửa hàng (store_cd).
df_receipt.groupby("store_cd").agg({"amount": "sum", "quantity": "sum"}).reset_index().head(10)
#(
    df_receipt.groupby("store_cd")
    .agg(
        sum_amount=("amount", "sum"),
        sum_quantity=("quantity", "sum")
    )
    .reset_index()
).head(10)

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


In [ ]:
#P-024: Từ dữ liệu biên nhận (df_receipt), hiển thị 10 bản ghi có ngày bán hàng (sales_ymd) mới nhất cho mỗi ID khách hàng (customer_id).

In [ ]:
#P-025: Từ dữ liệu biên nhận (df_receipt), hiển thị 10 bản ghi có ngày bán hàng (sales_ymd) cũ nhất cho mỗi ID khách hàng (customer_id).

In [ ]:
#P-026: Từ dữ liệu biên nhận (df_receipt), hiển thị 10 bản ghi có ngày bán hàng (sales_ymd) mới nhất và cũ nhất cho mỗi ID khách hàng (customer_id). Nếu ngày bán hàng khác nhau, hiển thị 10 bản ghi.

In [ ]:
#P-027: Từ dữ liệu biên nhận (df_receipt), tính và hiển thị trung bình số tiền bán hàng (amount) theo cửa hàng (store_cd), sắp xếp và hiển thị TOP 5.

In [ ]:
#P-028: Từ dữ liệu biên nhận (df_receipt), tính và hiển thị trung vị số tiền bán hàng (amount) theo cửa hàng (store_cd), sắp xếp và hiển thị TOP 5.

In [ ]:
#P-029: Từ dữ liệu biên nhận (df_receipt), hiển thị 10 bản ghi có giá trị mode lớn nhất của mã sản phẩm (product_cd) theo cửa hàng (store_cd).



In [ ]:
#P-030: Từ dữ liệu biên nhận (df_receipt), tính và hiển thị phân vị của số tiền bán hàng (amount) theo cửa hàng (store_cd), sắp xếp và hiển thị TOP 5.

In [ ]:
#P-031: Từ dữ liệu biên nhận (df_receipt), tính và hiển thị độ lệch chuẩn của số tiền bán hàng (amount) theo cửa hàng (store_cd), sắp xếp và hiển thị TOP 5.

In [ ]:
#P-032: Từ dữ liệu biên nhận (df_receipt), tính phần trăm phân vị (percentile) của số tiền bán hàng (amount) cho các cột tương ứng với giá trị 25%.

In [ ]:
#P-033: Từ dữ liệu biên nhận (df_receipt), tính và hiển thị số tiền bán hàng trung bình (amount) theo cửa hàng (store_cd), lọc ra các cửa hàng có trung bình lớn hơn hoặc bằng 330.



In [ ]:
#P-034: Từ dữ liệu biên nhận (df_receipt), tính và hiển thị số tiền bán hàng trung bình (amount) theo ID khách hàng (customer_id). Lưu ý loại bỏ các khách hàng có ID bắt đầu bằng 'Z' vì họ là nhân viên.

In [ ]:
#P-035: Từ dữ liệu biên nhận (df_receipt), tính và hiển thị số tiền bán hàng trung bình (amount) theo ID khách hàng (customer_id). Lưu ý loại bỏ các khách hàng có ID bắt đầu bằng 'Z' vì họ là nhân viên và chỉ hiển thị những người có số lượng bán hàng trung bình lớn hơn giá trị trung bình.

In [ ]:
#P-036: Kết hợp dữ liệu biên nhận (df_receipt) và dữ liệu cửa hàng (df_store), hiển thị 10 bản ghi đầu tiên bao gồm tất cả các cột từ dữ liệu biên nhận và tên cửa hàng (store_name) từ dữ liệu cửa hàng.

In [ ]:
#P-037: Kết hợp dữ liệu sản phẩm (df_product) và dữ liệu danh mục (df_category), hiển thị 10 bản ghi đầu tiên bao gồm tất cả các cột từ dữ liệu sản phẩm và tên danh mục (category_small_name) từ dữ liệu danh mục.

In [ ]:
#P-038: Từ dữ liệu khách hàng (df_customer) và dữ liệu chi tiết hóa đơn (df_receipt), hãy tính tổng số tiền bán hàng cho mỗi khách hàng và hiển thị 10 mục. Tuy nhiên, đối với khách hàng không có thành tích bán hàng, hiển thị tổng số tiền bán hàng là 0. Ngoài ra, chỉ xem xét khách hàng có mã giới tính (gender_cd) là nữ (1), và loại trừ khách hàng không phải là thành viên (khách hàng có ID khách hàng bắt đầu bằng "Z").

In [ ]:
#P-039: Từ dữ liệu chi tiết hóa đơn (df_receipt), hãy tạo dữ liệu của 20 khách hàng có số ngày bán hàng nhiều nhất và dữ liệu của 20 khách hàng có tổng số tiền bán hàng nhiều nhất, và sau outer join hai dữ liệu này. Tuy nhiên, loại trừ khách hàng không phải là thành viên (khách hàng có ID khách hàng bắt đầu bằng "Z").

In [ ]:
#P-040: Tạo dữ liệu kết hợp tất cả các cửa hàng và tất cả các sản phẩm (cross join). Hãy nhân trực tiếp dữ liệu cửa hàng (df_store) và dữ liệu sản phẩm (df_product) và tính số lượng.